In [2]:
# [1] 코스피 지수 구성종목 분석 페이지를 시가총액 순으로 긁어오자.

from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

def create_analysis_link(link):
    new = 'https://finance.naver.com' + link
    return 'https://finance.naver.com' + link[:6] + 'coinfo' + link[10:]


links = [] # 최종적인 링크들을 담을 테이블

# 시가총액 1~200위 페이지
for page in range(1,5):
    html = urlopen('https://finance.naver.com/sise/sise_market_sum.naver?&page='+str(page))
    bs = BeautifulSoup(html, 'html.parser')
    rankings = bs.findAll('a',{'href' : re.compile('^(/item/main.)')})

    # 각 종목별 페이지로 연결
    for ranking in rankings:
            if 'href' in ranking.attrs:
                midLink = str(create_analysis_link(ranking.attrs['href']))
                
                # 종목별 페이지 속의 '재무분석' 페이지로 연결
                html = urlopen("{}".format(midLink))
                bs = BeautifulSoup(html,'html.parser')
                link = bs.find('div', {'class' : 'section inner_sub'}).find('iframe')
                
                if 'src' in link.attrs:
                    financialLink = link.attrs['src']
                    links.append(financialLink)
                    
print(links[140])

https://navercomp.wisereport.co.kr/v2/company/c1010001.aspx?cmp_cd=004000


In [3]:
# [2] 어떤 자료들을 긁어올 것인지 '크롤링 모델'을 설정하자.

def unit_conversion(num):
    return round(float(num.replace(",","")))*10**8

class Content:
    
    #데이터 입력받기
    def __init__(self, bs_arr, sel_arr):
 
        # BeautifulSoup으로 긁어올 정보들 (추후에 정리)
        self.title = bs_arr[0]
        self.eps = bs_arr[1][0]
        self.bps = bs_arr[1][1]
        self.per = bs_arr[1][2]
        self.per_industry = bs_arr[1][3]
        self.pbr = bs_arr[1][4]
        self.dividend_rate = bs_arr[1][5]
    
        # Selenium으로 긁어올 정보들 (추후에 정리)
        self.market_cap = unit_conversion(sel_arr[0][:-2])
        self.per_average = round(sum(sel_arr[1])/len(sel_arr[1]) ,2)
        self.current_asset = unit_conversion(sel_arr[2])
        self.current_liability = unit_conversion(sel_arr[3])
        self.cashable_asset = unit_conversion(sel_arr[4])
        self.net_working_capital = self.current_asset-self.current_liability
        
    # 지금은 출력이지만, 이후에는 리스트화 시켜서 파일로 저장하기 용이하도록 만들 예정.
    def print(self):

        print("5년 PER 평균 : ", self.per_average)
        print("시가총액 : ", self.market_cap)
        print("유동자산 : ", self.current_asset)
        print("유동부채 : ", self.current_liability)
        print("현금 및 현금성 자산 : ", self.cashable_asset)
        print("순운전자본 : ", self.net_working_capital)

In [4]:
# [3] BeautifulSoup으로 긁어오기.

def make_name(arr):
    return arr[6][12:] + ' ' + 'ETF'

def extract_name(name):
    i = -1
    for i in range(1, len(name)):
        if name[-i] == '(':
            name = name[-i+1:-1]
            break
    return name

# writer.writerow(('순위', '종목 이름', 'EPS', 'BPS', 'PER', '업종PER', 'PBR', '현금배당수익률'))
# rank, title, basic data(리스트)

i=1
for one in links[:10]:
    html = urlopen("{}".format(one))
    bs = BeautifulSoup(html, 'html.parser')
    try:
        # title 가져오기
        title = bs.find('title').get_text()
        title = extract_name(title) # 형식에 맞게 수정
            
        # 재무정보 가져오기
        table = bs.findAll('table',{'class' : 'cmp-table'})[0]
        rows = table.findAll('tr')[2].findAll('dt')

        data = []
        for row in rows:
                
        # 필요없는 정보면 continue
            if row.get_text() == '12월 결산': 
                continue
                    
            temp = row.get_text().split()
            if len(temp) == 1:
                temp.append('N/A')
            data.append(temp[1])
                
    # ETF거나 등의 예외가 있을 때.        
    except IndexError as e:
        bs = bs.findAll('script', {'type':'text/javascript'})[2]
        arr = list(map(str,str(bs).split()))
            
        title = make_name(arr)
        data = ['N/A', 'N/A', 'N/A', 'N/A', 'N/A', 'N/A']
        
    finally:
        bs_arr = [i, title, data]
        print(bs_arr)
        i += 1

[1, '삼성전자', ['5,777', '43,611', '11.96', '10.64', '1.58', '2.09%']]
[2, 'LG에너지솔루션', ['3,963', '39,831', '108.01', '77.54', '10.75', 'N/A']]
[3, 'SK하이닉스', ['13,190', '90,394', '8.79', '10.64', '1.28', '1.33%']]
[4, 'NAVER', ['100,400', '157,753', '3.34', '5.94', '2.13', '0.15%']]
[5, '삼성바이오로직스', ['5,878', '74,537', '139.51', '83.29', '11.00', 'N/A']]
[6, '삼성전자', ['5,777', '43,611', '11.96', '10.64', '1.58', '2.09%']]
[7, '카카오', ['3,132', '23,024', '33.84', '5.94', '4.60', '0.05%']]
[8, '삼성SDI', ['16,621', '219,885', '35.32', '77.54', '2.67', '0.17%']]
[9, '현대차', ['17,846', '289,609', '9.97', '7.31', '0.61', '2.81%']]
[10, 'LG화학', ['46,880', '278,455', '11.16', '8.63', '1.88', '2.29%']]


In [5]:
# [4] Selenium으로 긁어오기.

from selenium import webdriver
from selenium.webdriver.common.by import By

import time

# headless 설정
options = webdriver.ChromeOptions()
options.add_argument('headless')

for one in links[:2]:
    # 브라우저 생성
    browser = webdriver.Chrome(chrome_options=options)
    browser.get('{}'.format(one))

    # 시가총액 구하기
    elementXpath = '//*[@summary="기업의 기본적인 시세정보(주가/전일대비/수익률,52주최고/최저,액면가,거래량/거래대금,시가총액,유동주식비율,외국인지분율,52주베타,수익률(1M/3M/6M/1Y))를 제공합니다."]/tbody/tr[5]/td'
    market_cap = browser.find_element(by=By.XPATH, value = elementXpath).text

    # per 평균 구하기
    browser.find_element(by=By.XPATH, value = '//*[@title="연간"]').click()

    elementXpath = '//table[@summary="주요재무정보를 제공합니다."][2]/tbody/tr[27]'
    element = browser.find_element(by=By.XPATH, value = elementXpath)
    per_arr = list(map(float,element.text.split()[1:6]))

    
    # 유동자산/유동부채 구하기
    browser.find_element_by_link_text("재무분석").click()
    browser.find_element_by_link_text("재무상태표").click()

    # Loading
    time.sleep(1)

    elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[2]/td[6]'
    current_asset = browser.find_element(by=By.XPATH, value = elementXpath).text

    elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[128]/td[6]'
    current_liability = browser.find_element(by=By.XPATH, value = elementXpath).text

    # 현금 및 현금성 자산 구하기
    elementXpath = '//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[47]/td[6]'
    cashable_asset = browser.find_element(by=By.XPATH, value = elementXpath).text

    # 객체화
    sel_arr = [market_cap, per_arr, current_asset, current_liability, cashable_asset]

    print(sel_arr)
    browser.quit()

    
# 문제점. 너무 오래 걸린다...
# 아무래도 셀레니움으로 200개 가량의 정보를 긁어오려면 하나씩 전부 이동하고 실행해야 하는데
# 이런 방식으로는 메모리에 계속 누적되어 실행이 느려지고
# 로딩이 1초 이내로 끝나지 않는 것 같다.
# 사실 나타날때까지 10초를 기다리는 설정도 해보았는데, 그것도 안되는 것을 보면
# 다른 문제이거나, 10초도 넘어간다는 이야기이다. 다만 두 번째부터도 per까지는 정상적으로 출력된다.

C:\Users\dck\AppData\Local\Temp/ipykernel_5344/1891685384.py:14: DeprecationWarning: use options instead of chrome_options
  browser = webdriver.Chrome(chrome_options=options)
C:\Users\dck\AppData\Local\Temp/ipykernel_5344/1891685384.py:30: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text("재무분석").click()
C:\Users\dck\AppData\Local\Temp/ipykernel_5344/1891685384.py:31: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  browser.find_element_by_link_text("재무상태표").click()


['4,125,120억원', [9.4, 6.42, 17.63, 21.09, 13.55], '2,181,631.9', '881,171.3', '390,314.2']


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//table[@summary="IFRS연결 연간 재무 정보를 제공합니다."][2]/tbody/tr[2]/td[6]"}
  (Session info: headless chrome=100.0.4896.60)
Stacktrace:
Backtrace:
	Ordinal0 [0x00617413+2389011]
	Ordinal0 [0x005A9F61+1941345]
	Ordinal0 [0x0049C658+837208]
	Ordinal0 [0x004C91DD+1020381]
	Ordinal0 [0x004C949B+1021083]
	Ordinal0 [0x004F6032+1204274]
	Ordinal0 [0x004E4194+1130900]
	Ordinal0 [0x004F4302+1196802]
	Ordinal0 [0x004E3F66+1130342]
	Ordinal0 [0x004BE546+976198]
	Ordinal0 [0x004BF456+980054]
	GetHandleVerifier [0x007C9632+1727522]
	GetHandleVerifier [0x0087BA4D+2457661]
	GetHandleVerifier [0x006AEB81+569713]
	GetHandleVerifier [0x006ADD76+566118]
	Ordinal0 [0x005B0B2B+1968939]
	Ordinal0 [0x005B5988+1989000]
	Ordinal0 [0x005B5A75+1989237]
	Ordinal0 [0x005BECB1+2026673]
	BaseThreadInitThunk [0x759EFA29+25]
	RtlGetAppContainerNamedObjectPath [0x775D7A7E+286]
	RtlGetAppContainerNamedObjectPath [0x775D7A4E+